In [11]:
import pandas as pd
import warnings
warnings.filterwarnings(
    "ignore",
    message="A value is trying to be set on a copy of a DataFrame",
    category=FutureWarning
)


In [5]:
# Load the notebook
records=pd.read_csv("./../data/InterviewTask/AI/records.csv")
gold_cases=pd.read_csv("./../data/InterviewTask/AI/gold_cases.csv")

In [40]:
len(records)

20000

## Claim Value Bande Based on Client Segment

In [29]:
gold_cases_p0=gold_cases['case_id'].loc[gold_cases['expected_priority']=="P0"].values


In [22]:
# records.loc[records['case_id'].isin( gold_cases_p0)]

In [34]:
gold_cases_p1=gold_cases['case_id'].loc[gold_cases['expected_priority']=="P1"].values

In [21]:
# records.loc[records['case_id'].isin( gold_cases_p1)]

In [36]:
gold_cases_p2=gold_cases['case_id'].loc[gold_cases['expected_priority']=="P2"].values

In [20]:
# records.loc[records['case_id'].isin( gold_cases_p2)]


In [38]:
gold_cases_p3=gold_cases['case_id'].loc[gold_cases['expected_priority']=="P3"].values

In [19]:
# records.loc[records['case_id'].isin( gold_cases_p3)]

# Process the data

In [44]:
records.columns

Index(['case_id', 'received_at', 'client_segment', 'jurisdiction',
       'service_line', 'claim_value_band', 'attachments_present',
       'free_text_summary', 'handler_notes', 'historical_outcome'],
      dtype='object')

In [45]:
#take the first 2000 rows
records_copy=records.loc[0:1500].copy()

#Drop columns ['received_at','free_text_summary', 'handler_notes', 'historical_outcome']
records_copy.drop(['received_at','free_text_summary', 'handler_notes'],axis=1,inplace=True)

#Replace null service_line with no data available
records_copy['service_line'].fillna("No data Available", inplace=True)

#Replace null client segment with no data available
records_copy['client_segment'].fillna("No data Available", inplace=True)

#Drop the cases without any case ids
records_copy.drop(records_copy.loc[records_copy["case_id"].isna()].index,axis=0,inplace=True)

#Replace null Jurisdiction with no data available
records_copy['jurisdiction'].fillna("No data Available", inplace=True)

#Replace the Null values in claim_value_band to Unknown
records_copy['claim_value_band'].fillna("Unknown", inplace=True)

#Replace Null values in the Attachment Present to False
records_copy['attachments_present'].fillna(False, inplace=True)

#Replace Null values in the historical_outcome to Unknown
records_copy['historical_outcome'].fillna("Unknown", inplace=True)

def covert_to_yesno(value):
    if value:
        return ("yes")
    else:
        return ("No")
#Convert Attachment Present to Yes No 
records_copy['attachments_present']=records_copy['attachments_present'].apply(covert_to_yesno)


In [46]:
import os
COLUMNS = ['case_id','severe_legal_or_regulatory_risk','business_critical_impact','potential_fraud','conflicting_information','complex_incident_details','policy_interpretation_issues','legal_disputes','jurisdictional_complexity','coverage_terms_unclear','exclusions_may_apply','new_or_unusual_claim_type','no_legal_or_fraud_concerns','unclear_incident_description','claim_invalid_or_fraudulent','required_conditions_not_met','risk_summary']


df_all_signals = pd.DataFrame(columns=COLUMNS)

for i in range(1,4):
    df=pd.read_csv(f"./../data/llm_out{i}.0.csv")
    df_all_signals = pd.concat( [df_all_signals, df],ignore_index=True)

In [47]:
#merge the two dataframes
processed_df=pd.merge(records_copy, df_all_signals, on="case_id", how="left")

In [ ]:
#risk mappings
class BaseRiskPriors:
    def __init__(self):
        self.YES_NO_MAP = {"Yes": 1.0,    
                           "No": 0.0,    
                           "No data available": 0.5}
        
        self.CLIENT_SEGMENT_RISK = {"SMB": 0.1,    
                                    "Mid-Market": 0.3,    
                                    "Enterprise": 0.5,    
                                    "No data Available": 0.3 }
        
        self.JURISDICTION_RISK   = {"UK": 0.2,    
                                    "EU": 0.3,      
                                    "US": 0.5,    
                                    "Other": 0.25,    
                                    "No data Available": 0.3 }
        
        self.SERVICE_LINE_MULTIPLIER = {"Legal": 1.3,    
                                        "Insurance": 1.0,    
                                        "Advisory": 0.7,    
                                        "No data Available": 1.0 }
        
        self.CLAIM_VALUE_RISK = {"<50k": 0.1,    
                                 "50k-250k": 0.3,    
                                 "250k-1m": 0.6,    
                                 ">1m": 0.9,    
                                 "Unknown": 0.4 }
        
        self.SIGNAL_WEIGHTS = {"severe_legal_or_regulatory_risk": 0.20,    
                                "business_critical_impact": 0.15,    
                                "potential_fraud": 0.15,
                                "legal_disputes": 0.15,
                                "policy_interpretation_issues": 0.10,
                                "jurisdictional_complexity": 0.10,
                                "coverage_terms_unclear": 0.10,
                                "exclusions_may_apply": 0.05
                                }
        



In [52]:
processed_df.columns

Index(['case_id', 'client_segment', 'jurisdiction', 'service_line',
       'claim_value_band', 'attachments_present', 'historical_outcome',
       'severe_legal_or_regulatory_risk', 'business_critical_impact',
       'potential_fraud', 'conflicting_information',
       'complex_incident_details', 'policy_interpretation_issues',
       'legal_disputes', 'jurisdictional_complexity', 'coverage_terms_unclear',
       'exclusions_may_apply', 'new_or_unusual_claim_type',
       'no_legal_or_fraud_concerns', 'unclear_incident_description',
       'claim_invalid_or_fraudulent', 'required_conditions_not_met',
       'risk_summary'],
      dtype='object')